# Load required libraries and the dataset

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Set ipython's max row display
pd.set_option('display.max_row', 200)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

pd.options.display.float_format = '{:.4f}'.format

In [2]:
# Read xlsx file for denied cases from Financial Year 17
df_17 = pd.read_excel("data/H-1B_Disclosure_Data_FY17.xlsx")
df_17 = df_17.loc[df_17.CASE_STATUS=='DENIED']
df_17.CASE_STATUS.value_counts()

DENIED    8480
Name: CASE_STATUS, dtype: int64

In [3]:
# Read xlsx file
df = pd.read_excel("data/H-1B_FY2018.xlsx")

In [4]:
# Finding any miss-match in columns
display(df.columns.difference(df_17.columns))
display(df_17.columns.difference(df.columns))

Index(['H-1B_DEPENDENT'], dtype='object')

Index(['H1B_DEPENDENT'], dtype='object')

In [5]:
# rename: "H-1B_DEPENDENT" to "H1B_DEPENDENT"
df.rename(columns={"H-1B_DEPENDENT" : "H1B_DEPENDENT"}, inplace=True)

In [6]:
# combine data-sets (row bind)
df = pd.concat([df,df_17])
del df_17

In [7]:
# View column names
df.columns

Index(['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE',
       'VISA_CLASS', 'EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE',
       'EMPLOYER_NAME', 'EMPLOYER_BUSINESS_DBA', 'EMPLOYER_ADDRESS',
       'EMPLOYER_CITY', 'EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE',
       'EMPLOYER_COUNTRY', 'EMPLOYER_PROVINCE', 'EMPLOYER_PHONE',
       'EMPLOYER_PHONE_EXT', 'AGENT_REPRESENTING_EMPLOYER',
       'AGENT_ATTORNEY_NAME', 'AGENT_ATTORNEY_CITY', 'AGENT_ATTORNEY_STATE',
       'JOB_TITLE', 'SOC_CODE', 'SOC_NAME', 'NAICS_CODE', 'TOTAL_WORKERS',
       'NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT', 'CHANGE_PREVIOUS_EMPLOYMENT',
       'NEW_CONCURRENT_EMPLOYMENT', 'CHANGE_EMPLOYER', 'AMENDED_PETITION',
       'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY',
       'PW_WAGE_LEVEL', 'PW_SOURCE', 'PW_SOURCE_YEAR', 'PW_SOURCE_OTHER',
       'WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY',
       'H1B_DEPENDENT', 'WILLFUL_VIOLATOR', 'SUPPORT_H1B', 'LABOR_CON_AGREE',

In [8]:
# Deatils of EMPLOYER and AGENT_ATTORNEY are ignored - (decison are un-biased)
# Select columns helpful for analysis
df = df[['CASE_SUBMITTED', 'DECISION_DATE','ORIGINAL_CERT_DATE',
       'VISA_CLASS', 'EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE',
       'AGENT_REPRESENTING_EMPLOYER','TOTAL_WORKERS',
       'NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT', 'CHANGE_PREVIOUS_EMPLOYMENT',
       'NEW_CONCURRENT_EMPLOYMENT', 'CHANGE_EMPLOYER', 'AMENDED_PETITION',
       'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY',
       'PW_WAGE_LEVEL', 'PW_SOURCE', 'PW_SOURCE_YEAR',
       'WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY',
       'H1B_DEPENDENT', 'WILLFUL_VIOLATOR', 'SUPPORT_H1B', 'LABOR_CON_AGREE',
       'EMPLOYER_CITY', 'WORKSITE_CITY', 'CASE_STATUS']]

---
# Data Transformation
---
## Re-defining Target (**CASE_STATUS**)

In [9]:
# View CASE_STATUS values
df.CASE_STATUS.value_counts()

CERTIFIED              80960
DENIED                 10040
CERTIFIED-WITHDRAWN     8900
WITHDRAWN               3202
Name: CASE_STATUS, dtype: int64

In [10]:
# Record with outcome - 'WITHDRAWN' is decided by employer and not by OFLC
# Drop records with CASE_STATUS 'WITHDRAWN'
df = df.query('CASE_STATUS != "WITHDRAWN"')
# CASE_STATUS values
df.CASE_STATUS.value_counts()

CERTIFIED              80960
DENIED                 10040
CERTIFIED-WITHDRAWN     8900
Name: CASE_STATUS, dtype: int64

In [11]:
# Records with outcome - 'CERTIFIED-WITHDRAWN' are 'CERTIFIED' by OFLC but. withdrawn by the employer (reason not revealed)
# Convert the DECISION_DATE of such cases to ORIGINAL_CERT_DATE (for consistency)
df['DECISION_DATE'] = df.apply(lambda x : x.ORIGINAL_CERT_DATE if x.CASE_STATUS == 'CERTIFIED-WITHDRAWN' else x.DECISION_DATE, axis = 1)
# Drop column ORIGINAL_CERT_DATE
df = df.drop(['ORIGINAL_CERT_DATE'],axis=1)

# Convert 'CERTIFIED-WITHDRAWN' to 'CERTIFIED'
df['CASE_STATUS'] = df.apply(lambda x : 'CERTIFIED' if x.CASE_STATUS == 'CERTIFIED-WITHDRAWN' else x.CASE_STATUS, axis = 1)

# CASE_STATUS values
df.CASE_STATUS.value_counts()/sum(df.CASE_STATUS.value_counts())

CERTIFIED   0.8995
DENIED      0.1005
Name: CASE_STATUS, dtype: float64

---
## Define a variable to keep track of invalid records

In [12]:
# Set default value to False 
df["INVALID"] = False

In [13]:
# EMPLOYMENT_START_DATE cannot be empty - all such cases are denied
df.loc[df.EMPLOYMENT_START_DATE.isnull(),"INVALID"] = True

In [14]:
# PREVAILING_WAGE cannot be zero or unusually high(>500000$) - all such cases are denied
df.loc[(df.PREVAILING_WAGE==0) | (df.PREVAILING_WAGE>500000),"INVALID"] = True

In [15]:
# PW_UNIT_OF_PAY cannot be empty - all such cases are denied
df.loc[df.PW_UNIT_OF_PAY.isnull(),"INVALID"] = True

In [16]:
# FULL_TIME_POSITION cannot be empty - all such cases are denied
df.loc[df.FULL_TIME_POSITION.isnull(),"INVALID"] = True

In [17]:
# PW_SOURCE (to verify PREVAILING_WAGE) cannot be empty - all such cases are denied
df.loc[df.PW_SOURCE.isnull(),"INVALID"] = True

In [18]:
# WORKSITE_CITY cannot be empty - all such cases are denied
df.loc[df.WORKSITE_CITY.isnull(),"INVALID"] = True

In [19]:
# All such invalid cases are denied
pd.crosstab(df.INVALID,df.CASE_STATUS)

CASE_STATUS,CERTIFIED,DENIED
INVALID,,
False,89860,9941
True,0,99


---
## Convert wage rates to a common units i.e, 'year' (per annum)

In [20]:
# defining a function for unit conversion
def convert_year(val, unit, full_time):
    '''
    val - value of wage
    unit - unit of wage
    '''
    if(val is np.NaN):
        return np.NaN
    if(unit is np.NaN):
        return 0 # NULL units were allotted PW value 0
    scale = {'Year':1, 'Month':12, 'Bi-Weekly':26, 'Week':52, 'Hour':2087}
    if(full_time == "N"):
        scale['Hour'] = 1820 # part-time are < 35hours/week - assumed 35
    return val*scale[unit]

In [21]:
# apply the conversion on PREVAILING_WAGE
df['PREVAILING_WAGE'] = df.apply(lambda x: convert_year(x.PREVAILING_WAGE,x.PW_UNIT_OF_PAY,x.FULL_TIME_POSITION), axis=1)
df['PW_UNIT_OF_PAY'] = 'Year' 

# apply the conversion on WAGE_RATE_OF_PAY_FROM and WAGE_RATE_OF_PAY_TO
df['WAGE_RATE_OF_PAY_FROM'] = df.apply(lambda x: convert_year(x.WAGE_RATE_OF_PAY_FROM,x.WAGE_UNIT_OF_PAY,x.FULL_TIME_POSITION), axis=1)
df['WAGE_RATE_OF_PAY_TO'] = df.apply(lambda x: convert_year(x.WAGE_RATE_OF_PAY_TO,x.WAGE_UNIT_OF_PAY,x.FULL_TIME_POSITION), axis=1)
df['WAGE_UNIT_OF_PAY'] = 'Year'

# if WAGE_RATE_OF_PAY_TO is zero -  replace with WAGE_RATE_OF_PAY_FROM 
df['WAGE_RATE_OF_PAY_TO'] = df.apply(lambda x : x.WAGE_RATE_OF_PAY_FROM if x.WAGE_RATE_OF_PAY_TO == 0 else x.WAGE_RATE_OF_PAY_TO, axis = 1)

In [22]:
# Unit of pay is no more required - as all are in years
df = df.drop(['PW_UNIT_OF_PAY','WAGE_UNIT_OF_PAY'],axis=1)

---
## Feature Engineering

In [23]:
# 1: BUFFER_DAYS = EMPLOYMENT_START_DATE - CASE_SUBMITTED (Days left to start employment)
df['BUFFER_DAYS'] = (df.EMPLOYMENT_START_DATE - df.CASE_SUBMITTED).dt.days
df['BUFFER_DAYS_BOOL'] = (df.EMPLOYMENT_START_DATE - df.CASE_SUBMITTED).dt.days>=0

# If no buffer - all cases denied except one
pd.crosstab(df.BUFFER_DAYS_BOOL,df.CASE_STATUS)

CASE_STATUS,CERTIFIED,DENIED
BUFFER_DAYS_BOOL,,
False,1,169
True,89859,9871


In [24]:
# 2: EMPLOYMENT_PERIOD(in days) = EMPLOYMENT_END_DATE - EMPLOYMENT_START_DATE
# EMPLOYMENT_PERIOD is not really saying anything - other than unusual observation (-183)
pd.crosstab((df.EMPLOYMENT_END_DATE - df.EMPLOYMENT_START_DATE).dt.days,df.CASE_STATUS)

CASE_STATUS,CERTIFIED,DENIED
row_0,,
-2558.0000,0,1
-183.0000,0,1
1.0000,3,2
2.0000,0,1
5.0000,0,1
6.0000,1,1
7.0000,2,0
9.0000,0,1
15.0000,1,2


In [25]:
# mark employment period < 0(makes no sense) and > 1096(max 3 years allowed) as invalid
emp_period = (df.EMPLOYMENT_END_DATE - df.EMPLOYMENT_START_DATE).dt.days
df.loc[(emp_period<0) | (emp_period>1096),"INVALID"] = True

In [26]:
# 3: ABOVE_PW(PREVAILING_WAGE) = WAGE_RATE_OF_PAY_FROM - PREVAILING_WAGE
df['ABOVE_PW'] = (df.WAGE_RATE_OF_PAY_FROM - df.PREVAILING_WAGE)>=0
pd.crosstab(df.ABOVE_PW,df.CASE_STATUS)

CASE_STATUS,CERTIFIED,DENIED
ABOVE_PW,,
False,15,2334
True,89845,7706


In [27]:
# Look at certified cases with WAGE_RATE_OF_PAY_FROM < PREVAILING_WAGE
temp = df.query('-310<(WAGE_RATE_OF_PAY_FROM - PREVAILING_WAGE)<0')
pd.crosstab(temp.WAGE_RATE_OF_PAY_FROM - temp.PREVAILING_WAGE,temp.CASE_STATUS)

CASE_STATUS,CERTIFIED,DENIED
row_0,,
-306.0000,0,1
-304.0000,0,1
-300.0000,0,2
-295.0000,0,1
-293.0000,0,1
-292.0000,0,1
-291.2000,0,1
-289.0000,0,1
-286.0000,0,1


In [28]:
# 4: PW_OLD(in years) = year(CASE_SUBMITTED) - PW_SOURCE_YEAR # cannot have negative values
pd.crosstab((df.CASE_SUBMITTED.dt.year - df.PW_SOURCE_YEAR),df.CASE_STATUS)

CASE_STATUS,CERTIFIED,DENIED
row_0,,
-90.0000,0,1
-1.0000,0,2
0.0000,84332,5468
1.0000,5331,4140
2.0000,91,337
3.0000,37,15
4.0000,17,5
5.0000,7,1
6.0000,39,0


In [29]:
# mark if age of PW_SOURCE in years < 0 (sourse from future - makes no sense) as invalid
df.loc[(df.CASE_SUBMITTED.dt.year - df.PW_SOURCE_YEAR)<0,"INVALID"] = True

In [30]:
# CASE_STATUS is decided within 12 days
pd.crosstab((df.DECISION_DATE - df.CASE_SUBMITTED),df.CASE_STATUS)

CASE_STATUS,CERTIFIED,DENIED
row_0,,
0 days,0,7
1 days,86,1724
2 days,180,2041
3 days,105,1167
4 days,12550,1598
5 days,2038,1475
6 days,52426,1351
7 days,22351,598
8 days,66,73


In [31]:
# View columns
df.columns

Index(['CASE_SUBMITTED', 'DECISION_DATE', 'VISA_CLASS',
       'EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE',
       'AGENT_REPRESENTING_EMPLOYER', 'TOTAL_WORKERS', 'NEW_EMPLOYMENT',
       'CONTINUED_EMPLOYMENT', 'CHANGE_PREVIOUS_EMPLOYMENT',
       'NEW_CONCURRENT_EMPLOYMENT', 'CHANGE_EMPLOYER', 'AMENDED_PETITION',
       'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_WAGE_LEVEL', 'PW_SOURCE',
       'PW_SOURCE_YEAR', 'WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO',
       'H1B_DEPENDENT', 'WILLFUL_VIOLATOR', 'SUPPORT_H1B', 'LABOR_CON_AGREE',
       'EMPLOYER_CITY', 'WORKSITE_CITY', 'CASE_STATUS', 'INVALID',
       'BUFFER_DAYS', 'BUFFER_DAYS_BOOL', 'ABOVE_PW'],
      dtype='object')

In [32]:
# Drop source attributes of extracted features
df = df.drop(['DECISION_DATE','CASE_SUBMITTED','PW_SOURCE_YEAR','PREVAILING_WAGE',
              'WAGE_RATE_OF_PAY_TO','EMPLOYMENT_START_DATE','EMPLOYMENT_END_DATE',
              'EMPLOYER_CITY', 'WORKSITE_CITY'],axis=1)

---
## Merge levels with low frequency

In [33]:
df.PW_SOURCE.value_counts()

OES      81184
Other    17915
CBA        711
DBA         18
SCA         16
Name: PW_SOURCE, dtype: int64

In [34]:
# Combine DBA and SCA to Other
df.loc[(df.PW_SOURCE=='DBA') | (df.PW_SOURCE=='SCA'),"PW_SOURCE"] = 'Other'

---
## Handling Missing Values

In [35]:
df.INVALID.value_counts()

False    99792
True       108
Name: INVALID, dtype: int64

In [36]:
# drop invalid rows
df = df[df.INVALID!=True]
df.drop(['INVALID'],inplace=True,axis=1)

In [37]:
df.isnull().sum()

VISA_CLASS                         0
AGENT_REPRESENTING_EMPLOYER     1477
TOTAL_WORKERS                      0
NEW_EMPLOYMENT                     0
CONTINUED_EMPLOYMENT               0
CHANGE_PREVIOUS_EMPLOYMENT         0
NEW_CONCURRENT_EMPLOYMENT          0
CHANGE_EMPLOYER                    0
AMENDED_PETITION                   0
FULL_TIME_POSITION                 0
PW_WAGE_LEVEL                   8022
PW_SOURCE                          0
WAGE_RATE_OF_PAY_FROM              0
H1B_DEPENDENT                   4382
WILLFUL_VIOLATOR                4383
SUPPORT_H1B                    64056
LABOR_CON_AGREE                64546
CASE_STATUS                        0
BUFFER_DAYS                        0
BUFFER_DAYS_BOOL                   0
ABOVE_PW                           0
dtype: int64

In [38]:
# Not requied - if invalid rows are dropped
# Central Impute (Assuming data missed entry)
#df.FULL_TIME_POSITION.fillna(df.AGENT_REPRESENTING_EMPLOYER.mode(),inplace=True)
#df.FULL_TIME_POSITION.fillna(df.FULL_TIME_POSITION.mode(),inplace=True)
#df.PW_SOURCE.fillna(df.PW_SOURCE.mode(),inplace=True)


# replace with a string 'missing'
df.fillna('missing',inplace=True)

df.isnull().sum()

VISA_CLASS                     0
AGENT_REPRESENTING_EMPLOYER    0
TOTAL_WORKERS                  0
NEW_EMPLOYMENT                 0
CONTINUED_EMPLOYMENT           0
CHANGE_PREVIOUS_EMPLOYMENT     0
NEW_CONCURRENT_EMPLOYMENT      0
CHANGE_EMPLOYER                0
AMENDED_PETITION               0
FULL_TIME_POSITION             0
PW_WAGE_LEVEL                  0
PW_SOURCE                      0
WAGE_RATE_OF_PAY_FROM          0
H1B_DEPENDENT                  0
WILLFUL_VIOLATOR               0
SUPPORT_H1B                    0
LABOR_CON_AGREE                0
CASE_STATUS                    0
BUFFER_DAYS                    0
BUFFER_DAYS_BOOL               0
ABOVE_PW                       0
dtype: int64

In [39]:
# Save to local
df.to_csv("data/dataCleaned.csv",index=False)